In [ ]:
!pip install inltk

     |████████████████████████████████| 235kB 2.8MB/s 
     |████████████████████████████████| 81kB 7.1MB/s 
     |████████████████████████████████| 1.2MB 12.1MB/s 
     |████████████████████████████████| 1.3MB 18.5MB/s 
     |████████████████████████████████| 143kB 20.3MB/s 
     |████████████████████████████████| 296kB 20.5MB/s 
  Created wheel for typing: filename=typing-3.7.4.3-cp37-none-any.whl size=26308 sha256=6f0491bc4a261d4fef6cc85b033c79a30938325127caf25f603ad422282ca8ab
  Stored in directory: /root/.cache/pip/wheels/2d/04/41/8e1836e79581989c22eebac3f4e70aaac9af07b0908da173be
Successfully built typing
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


In [ ]:
import os
import re
import spacy
import nltk
import numpy as np
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from scipy.sparse import csr_matrix
from sklearn import metrics
from inltk.inltk import setup
from inltk.inltk import get_embedding_vectors
setup('hi-en') #this line will return error but the subsequent steps will work without issues if not run this a couple of times

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
os.chdir('/gdrive/My Drive/drivebuddy_text_classification')

In [ ]:
def filter_query(query):
    query = query.lower()
    query = re.sub(r'[@][^\s]+', '', query)
    query = re.sub(r'pav.{0,3}bhaji', ' pavbhaji ', query)
    query = re.sub(r'[\!-\/\:-\@]+', ' ', query)
    query = re.sub('[^A-Za-z0-9\s]+', ' ', query)
    query = re.sub(r'[\t\n\r\f ]+', ' ', re.sub(r'\.', '. ', query))
    query = ' '.join([w for w in query.split() if w not in stopwords.words('english')])
    
    # print (query)
    # doc = nlp(query)
    # tokens = [lemmatizer.lemmatize(t) for t in tokens]
    # filt_q = ' '.join(tokens)
    filt_q = re.sub(r'\b(n\'t|nt)\b', 'not', query)
    filt_q = re.sub(r'\'ll\b', 'will', filt_q)
    return filt_q

In [ ]:
import json

In [ ]:
with open('dataset_mod/pavbhaji.json', 'r') as f:
    data = json.load(f)

In [ ]:
indexed_data = {d['display_url'].split('/')[-1]: d['edge_media_to_caption']['edges'][0]['node']['text'] for d in data if d['edge_media_to_caption']['edges']}

In [ ]:
file_names0_set = set([f.split('/')[-1] for f in glob.glob('dataset_mod/images/0/*.jpg')])
file_names1_set = set([f.split('/')[-1] for f in glob.glob('dataset_mod/images/1/*.jpg')])

In [ ]:
data_with_labels = pd.DataFrame([{'name': name, 'text': indexed_data[name], 'label': 1 if name in file_names1_set else 0} for name in file_names0_set | file_names1_set])

In [ ]:
processed_text = data_with_labels['text'].map(filter_query)
df = pd.DataFrame({'name':data_with_labels['name'], 'text':processed_text, 'label': data_with_labels['label']})

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2)

In [ ]:
from functools import partial
from inltk.inltk import setup

# setup('hi-en')

from inltk.inltk import tokenize
tokenize_hing = partial(tokenize, language_code='hi-en')  # tokenizer for hindi-english


In [ ]:
from inltk.inltk import get_embedding_vectors

In [ ]:
import tensorflow as tf
# import tensorflow_hub as hub
import numpy as np
import os
import pandas as pd
import re
import json

from tensorflow import keras
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Lambda, Layer, Masking
from tensorflow.keras.models import Model, load_model, model_from_json
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, Callback
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow import keras
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Lambda, Bidirectional
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow_hub as hub


In [ ]:
# define a function to get embeddings matrix from plain text
def get_embeddings(text):
    emb_list = get_embedding_vectors(text, 'hi-en')
    if emb_list:
        emb_matrix = np.vstack(emb_list)
    else:
        emb_matrix = np.zeros((1,400))
    return emb_matrix

In [ ]:
# get embeddings for all training and testing documents

X_train_input_matrices = [get_embeddings(x) for x in X_train]

X_test_input_matrices = [get_embeddings(x) for x in X_test]

/usr/local/lib/python3.7/dist-packages/torch/serialization.py:656: SourceChangeWarning: source code of class 'torch.nn.modules.loss.CrossEntropyLoss' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:656: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:656: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the original source code by accessing the object's source attrib

In [ ]:
max(max([t.shape[0] for t in X_train_input_matrices]), max([t.shape[0] for t in X_test_input_matrices]))

(280, 147)

In [ ]:
embed_size = 400
max_length = max(max([t.shape[0] for t in X_train_input_matrices]), max([t.shape[0] for t in X_test_input_matrices]))  # maximum length of document

# convert list of matrices to 3D matrix
X_train_input_matrix = np.zeros([len(X_train_input_matrices), max_length, embed_size])
for i, mat in enumerate(X_train_input_matrices):
    X_train_input_matrix[i, :mat.shape[0], :mat.shape[1]] = mat

X_test_input_matrix = np.zeros([len(X_test_input_matrices), max_length, embed_size])
for i, mat in enumerate(X_test_input_matrices):
    X_test_input_matrix[i, :mat.shape[0], :mat.shape[1]] = mat


In [ ]:
X_train_input_matrix[0].shape

(280, 400)

In [ ]:
# defining bidirectional lstm model with dense network at the top

# embed_size = 400
# max_length = 387

model2 = Sequential()
# model2.add(Input(shape=(None, 400), dtype=tf.float32))
model2.add(Masking(mask_value=0., input_shape=(None, embed_size), ))
# model2.add(Lambda(get_embeddings, output_shape=(None, 400), dtype=tf.float32))
# model2.add(Embedding(vocab_size, 100, name="embedding", trainable=False))
model2.add(Bidirectional(LSTM(64, unit_forget_bias=True, name="lstm", dropout=0.5)))
model2.add(Dense(128, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
model2.summary()

LEARNING_RATE = 0.001

optimizer = Adam(lr=LEARNING_RATE)

model2.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['binary_accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 400)         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               238080    
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 254,721
Trainable params: 254,721
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping


class LearningRateTracker(Callback):
    def on_epoch_end(self, epoch, logs=None):
        print(" - lr: {}".format(K.eval(self.model.optimizer.lr))) 
# session.run(tf.global_variables_initializer())
# session.run(tf.tables_initializer())

LR_PATIENCE = 10
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=LR_PATIENCE, min_lr=1e-8, verbose=1, mode="min")
es_callback = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
lr_tracker = LearningRateTracker()

history = model2.fit(X_train_input_matrix, 
          y_train,
          validation_data=(X_test_input_matrix, y_test),
          epochs=1000,
          batch_size=256,
          callbacks=[es_callback, lr_tracker, reduce_lr])
model2.save_weights('./model_cust_lstm.h5')

# achieved maximum accuracy of 72%

Epoch 1/1000
2/2 [==============================] - 14s 4s/step - loss: 0.7046 - binary_accuracy: 0.4648 - val_loss: 0.6917 - val_binary_accuracy: 0.5714
 - lr: 0.0010000000474974513
Epoch 2/1000
2/2 [==============================] - 5s 2s/step - loss: 0.6851 - binary_accuracy: 0.5574 - val_loss: 0.6813 - val_binary_accuracy: 0.6374
 - lr: 0.0010000000474974513
Epoch 3/1000
2/2 [==============================] - 4s 2s/step - loss: 0.6734 - binary_accuracy: 0.6157 - val_loss: 0.6728 - val_binary_accuracy: 0.6374
 - lr: 0.0010000000474974513
Epoch 4/1000
2/2 [==============================] - 4s 2s/step - loss: 0.6663 - binary_accuracy: 0.6175 - val_loss: 0.6656 - val_binary_accuracy: 0.5714
 - lr: 0.0010000000474974513
Epoch 5/1000
2/2 [==============================] - 4s 2s/step - loss: 0.6550 - binary_accuracy: 0.5865 - val_loss: 0.6617 - val_binary_accuracy: 0.5824
 - lr: 0.0010000000474974513
Epoch 6/1000
2/2 [==============================] - 4s 2s/step - loss: 0.6574 - binary_ac

In [ ]:
predictions = model2.predict_classes(X_test_input_matrix)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
print (predictions)
print("Accuracy:",metrics.accuracy_score(y_test, predictions))

[[0]
 [0]
 [0]
 [0]
 ...
 [1]
 [1]
 [1]
 [1]]
Accuracy: 0.6703296703296703
